# Iris Flower Classifer
We are going to classify of flowers by species. The dataset which we will use for this toy DNN is the [Iris flower data set](https://en.wikipedia.org/wiki/Iris_flower_data_set). This is a multivariate dataset from 1936 - Ronald Fisher. There are 50 samples for each of the 3 species of Iris ((Iris setosa, Iris virginica and Iris versicolor). There are 4 features for each sample measured in cm:
- length of sepals
- width of sepals
- length of petals
- width of petals

Tensorflow provides a [reference](https://www.tensorflow.org/get_started/estimator) for this project.

In [ ]:
# Dependancies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import time
import numpy as np
import urllib.request

In [ ]:
# Configure TF Logging
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
# Directories
model_dir_root = '/tmp/iris'
model_dir = os.path.join(model_dir_root, "deep_" + str(int(time.time())))

In [ ]:
# Hyper Params
batch_size = 10
num_steps = 2000

In [ ]:
# Load Dataset
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"
IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [ ]:
def loadDataset ():
    # If the training and test sets aren't stored locally, download them
    if not os.path.exists(IRIS_TRAINING):
        with urllib.request.urlopen(IRIS_TRAINING_URL) as url: 
            raw = url.read()
        with open(IRIS_TRAINING, "wb") as f:
            f.write(raw)

    if not os.path.exists(IRIS_TEST):
        with urllib.request.urlopen(IRIS_TEST_URL) as url: 
            raw = url.read()
        with open(IRIS_TEST, "wb") as f:
            f.write(raw)
            
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TRAINING,
        target_dtype=np.int,
        features_dtype=np.float32)
    
    test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TEST,
        target_dtype=np.int,
        features_dtype=np.float32)
    
    return training_set, test_set

In [ ]:
def log_data(dataset, example, all=False):
    if all == True:
        # log the training dataset
        print(dataset)

    # log 1 example and 1 answer
    print("X: {}".format(dataset[0][example]))
    print("Y: {}".format(dataset[1][example]))

In [ ]:
# Build input function
def generate_input_fn(dataset, batch_size=batch_size):
    def _input_fn():
        X = tf.constant(dataset[0])
        Y = tf.constant(dataset[1], dtype=tf.int32)
        
        X_batch, Y_batch = tf.train.shuffle_batch(
            [X,Y],
            batch_size=batch_size,
            capacity=8*batch_size,
            min_after_dequeue=4*batch_size,
            enqueue_many=True
        )
        
        return {'features': X_batch}, Y_batch
    return _input_fn

In [ ]:
# Build classifier
def define_and_run_dnn_classifier(num_steps, logdir, lr=.1, batch_size=batch_size):
    feature_columns = [tf.contrib.layers.real_valued_column('features', dimension=4)]

    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=3,
        hidden_units=[10,10,10],
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
    )
    
    # Train classifer
    classifier.train(
        input_fn=generate_input_fn(
            training_set,
            batch_size=batch_size
        ),
        steps=num_steps
    )
    
    print("Finished running the deep training")
    print("evaluating DNN classifier accuracy")
    
    # Test classifer
    accuracy_score = classifier.evaluate(
        input_fn=generate_input_fn(
            test_set,
            batch_size=batch_size
        ),
        steps=100
    )['accuracy']
    
    print("DNN classifier accuracy: {0:f}".format(accuracy_score))
    
    # Make a prediction
    predictions = classifier.predict(
        input_fn=generate_input_fn(
            test_set,
            batch_size=1
        )
    )
    
    # make a prediction
    print("DNN classifier prediction: ")
    for i in range(10):
        prediction = predictions.__next__()['probabilities']
        print("\nPrediction for example {0}: {1}".format(i, np.argmax(prediction)))
        log_data(test_set, i)
        

In [ ]:
# Run classifer
training_set, test_set = loadDataset()

# log an example
log_data(training_set, 0)
        

In [ ]:
print("Running DNN Classifier")
define_and_run_dnn_classifier(
    num_steps,
    model_dir
)

In [ ]:
# tensorboard --logdir=/tmp/iris